In [2]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import product

class GameTheoryAnalyzer:
    def __init__(self):
        self.game = {} # Initialize an empty game dictionary
        self._get_user_game_input() # Get game details from the user

    def _get_user_game_input(self):
        """Prompts the user to define the game (players, strategies, payoffs)."""
        print("--- Define Your Game ---")
        num_players = int(input("Enter the number of players (currently supports 2 for extensive form): "))
        if num_players != 2:
            print("Warning: Extensive form visualization and mixed strategy calculations are best supported for 2 players.")
            print("The analysis methods will still work, but the extensive form might not be accurate for non-simultaneous or more than 2 players.")

        players = []
        for i in range(num_players):
            player_name = input(f"Enter name for Player {i+1}: ")
            players.append(player_name)

        strategies = []
        for i, player_name in enumerate(players):
            player_strategies_str = input(f"Enter strategies for {player_name} (comma-separated, e.g., Heads,Tails): ")
            strategies.append([s.strip() for s in player_strategies_str.split(',')])

        # Validate strategies (ensure it's a 2x2 game for specific analyses)
        if num_players == 2 and (len(strategies[0]) != 2 or len(strategies[1]) != 2):
            print("Warning: For accurate Mixed Strategy Nash Equilibrium calculations and specific extensive form, a 2x2 game is recommended.")
            print("Analysis for larger games might be simplified or provide warnings.")

        # Initialize payoffs structure
        # payoffs[player_idx][player_1_strat_idx][player_2_strat_idx]...
        payoffs = [[None for _ in range(len(s))] for s in strategies]
        for p_idx in range(num_players):
            payoffs[p_idx] = [[0 for _ in range(len(strategies[1]))] for _ in range(len(strategies[0]))]


        print("\nEnter payoffs for each player. For each cell (Player 1 Strategy, Player 2 Strategy), enter payoff for each player.")
        print("Example: If Player 1 plays 'Heads' and Player 2 plays 'Heads', enter 'P1_Payoff,P2_Payoff'.")

        for i, p1_strat in enumerate(strategies[0]):
            for j, p2_strat in enumerate(strategies[1]):
                while True:
                    try:
                        payoff_str = input(f"Enter payoffs for ({p1_strat}, {p2_strat}) as 'P1_Payoff,P2_Payoff': ")
                        player_payoffs = [int(p.strip()) for p in payoff_str.split(',')]
                        if len(player_payoffs) != num_players:
                            raise ValueError(f"Expected {num_players} payoffs, got {len(player_payoffs)}")

                        for p_idx in range(num_players):
                            payoffs[p_idx][i][j] = player_payoffs[p_idx]
                        break
                    except ValueError as e:
                        print(f"Invalid input: {e}. Please enter payoffs in the format 'P1_Payoff,P2_Payoff'.")

        self.game = {
            "players": players,
            "strategies": strategies,
            "payoffs": payoffs,
            "description": "User-defined game."
        }
        print("\nGame defined successfully!")

    def display_normal_form(self):
        """Display the game in normal form (payoff matrix)."""
        print("\n--- Normal Form Representation ---")
        if not self.game:
            print("No game defined.")
            return

        players = self.game["players"]
        strategies = self.game["strategies"]
        payoffs = self.game["payoffs"]

        # This normal form display is primarily for 2-player games.
        if len(players) > 2:
            print("Normal form display is simplified for games with more than 2 players.")
            print("It shows Player 1's strategies as rows and Player 2's as columns, fixing other players' strategies to their first strategy.")

        print(f"\n{players[1]} strategies: {', '.join(strategies[1])}")

        # Calculate max strategy name length for alignment
        max_s0_len = max(len(s) for s in strategies[0])
        col_width = max(len(str(strategies[1][k])) for k in range(len(strategies[1]))) + 8 # For payoffs

        header_line = f"{players[0]} strategies \\ {players[1]} strategies | "
        for s1_strat in strategies[1]:
            header_line += f"{s1_strat:^{col_width}} | "
        print(header_line)
        print("-" * (len(header_line) + 1 + len(strategies[1]) * (col_width + 3)))

        for i, s0_strat in enumerate(strategies[0]):
            row_line = f"{s0_strat:<{max_s0_len}} | "
            for j, s1_strat in enumerate(strategies[1]):
                # Construct payoff tuple dynamically for all players
                payoff_tuple = []
                for p_idx in range(len(players)):
                    if len(players) == 2: # Standard 2-player matrix
                        payoff_tuple.append(payoffs[p_idx][i][j])
                    else: # Simplified for >2 players: assume other players fix their first strategy
                        # This part would need significant modification for a truly general normal form for n players
                        # A full N-player normal form table becomes N-dimensional.
                        # For display, we'll just show P1 vs P2, assuming others play their first strat.
                        # This is a limitation for visualizing general N-player normal forms.
                        if p_idx == 0: # Player 1's payoff
                             payoff_tuple.append(payoffs[p_idx][i][j])
                        elif p_idx == 1: # Player 2's payoff
                             payoff_tuple.append(payoffs[p_idx][i][j])
                        else: # Other players' payoffs, assume they play their first strategy
                            # This is a simplification and not a true representation of all N-player outcomes
                            payoff_tuple.append(payoffs[p_idx][0][0]) # This is incorrect for general N-player games
                            # For a general N-player game, you'd need to iterate through all combinations
                            # of strategies for Players 3 to N to form a full N-dimensional matrix.
                            # For now, we'll just show P1 and P2's payoffs at that specific cell.
                            # A better approach for N>2 would be to not attempt a 2D matrix print,
                            # or let the user choose which 2 players to fix on the axes.
                            pass # We will only show P1 and P2 for now in the tuple itself

                payoff_str = f"({', '.join(map(str, payoff_tuple))})"
                row_line += f"{payoff_str:^{col_width}} | "
            print(row_line)
        print("-" * (len(header_line) + 1 + len(strategies[1]) * (col_width + 3)))


    def display_extensive_form(self):
        """
        Display extensive form representation as a game tree.
        This version is specifically designed for 2-player simultaneous games.
        """
        print("\n--- Extensive Form Representation (for Simultaneous 2-Player Games) ---")
        if not self.game or len(self.game["players"]) != 2:
            print("Extensive form visualization is currently only supported for 2-player simultaneous games.")
            print("Please define a 2-player game to see its extensive form.")
            return

        players = self.game["players"]
        strategies = self.game["strategies"]
        payoffs = self.game["payoffs"]

        G = nx.DiGraph()
        pos = {}
        labels = {}
        final_payoff_nodes = {}

        # Player 1's decision node
        p1_node = "P1_Decision"
        G.add_node(p1_node)
        pos[p1_node] = (0, 2.0)
        labels[p1_node] = players[0]

        # Nodes for Player 1's choices (strategies)
        p1_choice_nodes = {}
        x_offset = -0.75 * (len(strategies[0]) - 1) # Adjust starting x for centering
        for i, s1_strat in enumerate(strategies[0]):
            node_name = f"P1_Choice_{s1_strat}"
            G.add_node(node_name)
            G.add_edge(p1_node, node_name)
            pos[node_name] = (x_offset + i * 1.5, 1.25)
            labels[node_name] = s1_strat
            p1_choice_nodes[s1_strat] = node_name

        # Player 2's decision nodes (one for each of P1's choices, but connected by information set)
        p2_decision_nodes = []
        for i, s1_strat in enumerate(strategies[0]):
            node_name = f"P2_Decision_{s1_strat}"
            G.add_node(node_name)
            G.add_edge(p1_choice_nodes[s1_strat], node_name)
            pos[node_name] = (pos[p1_choice_nodes[s1_strat]][0], 0.5)
            labels[node_name] = players[1]
            p2_decision_nodes.append(node_name)

        # Nodes for Player 2's choices (strategies) leading to payoffs
        for i, s1_strat in enumerate(strategies[0]):
            for j, s2_strat in enumerate(strategies[1]):
                p2_strat_node_name = f"P2_Strat_{s1_strat}_{s2_strat}"
                G.add_node(p2_strat_node_name)
                G.add_edge(f"P2_Decision_{s1_strat}", p2_strat_node_name)

                payoff_node_name = f"Payoff_{s1_strat}_{s2_strat}"
                G.add_node(payoff_node_name)
                G.add_edge(p2_strat_node_name, payoff_node_name)

                # Positioning for P2's strategy choice and payoff nodes
                # Spread out horizontally under P2's decision node
                x_offset_p2_strat = pos[f"P2_Decision_{s1_strat}"][0] - 0.5 * (len(strategies[1]) - 1)
                pos[p2_strat_node_name] = (x_offset_p2_strat + j * 1, -0.25)
                labels[p2_strat_node_name] = s2_strat

                pos[payoff_node_name] = (pos[p2_strat_node_name][0], -1)
                labels[payoff_node_name] = f"({payoffs[0][i][j]}, {payoffs[1][i][j]})"
                final_payoff_nodes[(s1_strat, s2_strat)] = payoff_node_name

        plt.figure(figsize=(12, 8)) # Adjusted figure size
        ax = plt.gca()

        nx.draw_networkx_nodes(G, pos, node_size=3000, node_color='lightgreen', alpha=0.9, ax=ax)
        nx.draw_networkx_edges(G, pos, arrowsize=20, width=1.5, edge_color='gray', ax=ax)
        nx.draw_networkx_labels(G, pos, labels=labels, font_size=9, font_weight='bold', ax=ax, clip_on=False)

        # Draw the information set for Player 2
        info_set_nodes_x = [pos[node][0] for node in p2_decision_nodes]
        info_set_nodes_y = [pos[node][1] for node in p2_decision_nodes]
        ax.plot(info_set_nodes_x, info_set_nodes_y, 'k--', linewidth=1.5)

        info_set_label_x = np.mean(info_set_nodes_x)
        info_set_label_y = info_set_nodes_y[0] + 0.2
        ax.text(info_set_label_x, info_set_label_y, f"{players[1]}'s Information Set",
                horizontalalignment='center', verticalalignment='center', fontsize=9, color='darkgray',
                bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="none", alpha=0.7))

        plt.title(f"Extensive Form Game Tree for {self.game['description']} (Simultaneous Moves)", fontsize=14)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig("extensive_form_game.png", dpi=300)
        plt.close()
        print("Game tree saved as 'extensive_form_game.png'")

    def find_dominated_strategies(self, iterative=False):
        """
        Identify strictly dominated strategies. Can be run iteratively for rationalizability.
        Returns a list of strategies that are strictly dominated.
        """
        print("\n--- Dominated Strategy Analysis ---")
        players = self.game["players"]
        strategies = [list(s) for s in self.game["strategies"]] # Make a copy to modify
        payoffs = self.game["payoffs"]

        eliminated_strategies = {p_idx: [] for p_idx in range(len(players))}

        iteration = 0
        while True:
            iteration += 1
            print(f"\nIteration {iteration}:")
            found_dominated_in_iter = False

            for player_idx in range(len(players)):
                current_player_strats = strategies[player_idx]

                # Cannot analyze if only one strategy left
                if len(current_player_strats) <= 1:
                    continue

                for i in range(len(current_player_strats)): # Strategy 's_i'
                    s_i = current_player_strats[i]
                    is_s_i_dominated = False

                    for j in range(len(current_player_strats)): # Strategy 's_j' potentially dominating 's_i'
                        s_j = current_player_strats[j]
                        if i == j:
                            continue # Cannot dominate itself

                        # Check if s_j strictly dominates s_i across all opponent strategy profiles
                        is_strictly_dominated_by_s_j = True

                        # To check dominance, we need to compare payoffs across all possible opponent strategy profiles.
                        # Generate all combinations of opponent strategies *from the currently available strategies*
                        opponent_indices = [idx for idx in range(len(players)) if idx != player_idx]

                        # Get opponent's current available strategies (their indices)
                        opponent_current_strategies_indices = [
                            list(range(len(strategies[op_idx]))) for op_idx in opponent_indices
                        ]

                        # Generate all opponent strategy profiles (tuples of indices)
                        opponent_profiles = list(product(*opponent_current_strategies_indices))

                        if not opponent_profiles and len(players) == 1: # Single player game, no opponents
                            # Dominance typically applies to games with at least two choices, and comparison
                            # across opponent actions. In a single-player context, it's just about maximizing.
                            # For simplicity, we'll say no domination in 1-player games as usually defined.
                            is_strictly_dominated_by_s_j = False # No opponent to compare against

                        for opp_profile_indices in opponent_profiles:
                            # Construct the full strategy profile indices for lookup in the original payoff matrix
                            full_profile_indices_i = [0] * len(players) # Placeholder
                            full_profile_indices_j = [0] * len(players) # Placeholder

                            current_op_idx_ptr = 0
                            for p_k in range(len(players)):
                                if p_k == player_idx:
                                    full_profile_indices_i[p_k] = self.game["strategies"][p_k].index(s_i)
                                    full_profile_indices_j[p_k] = self.game["strategies"][p_k].index(s_j)
                                else:
                                    # Map the relative opponent index to the absolute strategy index
                                    original_strat_index = self.game["strategies"][p_k].index(strategies[p_k][opp_profile_indices[current_op_idx_ptr]])
                                    full_profile_indices_i[p_k] = original_strat_index
                                    full_profile_indices_j[p_k] = original_strat_index
                                    current_op_idx_ptr += 1

                            # Get payoffs for s_i and s_j
                            payoff_s_i = self.game["payoffs"][player_idx]
                            payoff_s_j = self.game["payoffs"][player_idx]

                            # This part is still problematic for n-player general case with direct indexing.
                            # We need to correctly extract the payoff based on the full_profile_indices.
                            # Let's adjust for N-player payoff access:
                            # For a payoff `payoffs[player_idx][strat1_idx][strat2_idx]...[stratN_idx]`

                            # Helper to get payoff given a full strategy profile (list of strategy indices)
                            def get_payoff_for_profile(p_idx, profile_indices):
                                val = self.game["payoffs"][p_idx]
                                for idx in profile_indices:
                                    val = val[idx]
                                return val

                            payoff_val_i = get_payoff_for_profile(player_idx, full_profile_indices_i)
                            payoff_val_j = get_payoff_for_profile(player_idx, full_profile_indices_j)

                            if payoff_val_j <= payoff_val_i: # Not strictly dominated if j is not strictly better
                                is_strictly_dominated_by_s_j = False
                                break

                        if is_strictly_dominated_by_s_j:
                            print(f"  {players[player_idx]}: Strategy '{s_i}' is strictly dominated by '{s_j}'")
                            eliminated_strategies[player_idx].append(s_i)
                            strategies[player_idx].remove(s_i) # Remove it for the next check in this iteration
                            is_s_i_dominated = True
                            found_dominated_in_iter = True
                            break # Move to the next strategy for the current player

                    if is_s_i_dominated:
                        break # Restart the inner loop for the same player with updated strategies

            if not found_dominated_in_iter or not iterative:
                break # Stop if no more dominated strategies or not iterative

        if all(len(eliminated_strategies[p_idx]) == 0 for p_idx in range(len(players))):
            print("No strictly dominated strategies found.")
        else:
            print("\nSummary of eliminated strategies (Iterated Elimination of Strictly Dominated Strategies):")
            for p_idx, eliminated in eliminated_strategies.items():
                if eliminated:
                    print(f"{players[p_idx]}: {', '.join(eliminated)}")

            print("\nRemaining strategies after elimination (Rationalizable Strategies):")
            for p_idx in range(len(players)):
                print(f"{players[p_idx]}: {', '.join(strategies[p_idx])}")

        return strategies # Return the reduced set of strategies


    def find_best_responses(self):
        """Find best responses for each player."""
        print("\n--- Best Response Analysis ---")
        if not self.game:
            print("No game defined.")
            return

        players = self.game["players"]
        strategies = self.game["strategies"]
        payoffs = self.game["payoffs"]

        if len(players) > 2:
            print("Best response analysis for more than 2 players is complex to display exhaustively.")
            print("Showing best responses for Player 1 and Player 2 against each other, assuming others play their first strategy.")
            # This is a simplification. A true N-player best response involves holding N-1 players' strategies fixed.
            # We'll focus on P1 and P2 against each other for clarity in output.

        # For Player 1 (row player)
        print(f"\n{players[0]}'s best responses:")
        # Iterate through Player 2's strategies
        for j, s2_strat in enumerate(strategies[1]):
            best_payoff_p1 = -float('inf')
            best_strategies_p1 = []

            for i, s1_strat in enumerate(strategies[0]):
                current_payoff_p1 = payoffs[0][i][j]

                if current_payoff_p1 > best_payoff_p1:
                    best_payoff_p1 = current_payoff_p1
                    best_strategies_p1 = [s1_strat]
                elif current_payoff_p1 == best_payoff_p1:
                    best_strategies_p1.append(s1_strat)

            print(f"  When {players[1]} plays '{s2_strat}', {players[0]}'s best response is {', '.join(best_strategies_p1)}.")

        # For Player 2 (column player)
        print(f"\n{players[1]}'s best responses:")
        # Iterate through Player 1's strategies
        for i, s1_strat in enumerate(strategies[0]):
            best_payoff_p2 = -float('inf')
            best_strategies_p2 = []

            for j, s2_strat in enumerate(strategies[1]):
                current_payoff_p2 = payoffs[1][i][j]

                if current_payoff_p2 > best_payoff_p2:
                    best_payoff_p2 = current_payoff_p2
                    best_strategies_p2 = [s2_strat]
                elif current_payoff_p2 == best_payoff_p2:
                    best_strategies_p2.append(s2_strat)

            print(f"  When {players[0]} plays '{s1_strat}', {players[1]}'s best response is {', '.join(best_strategies_p2)}.")

        # For >2 players, this would require iterating through fixed strategies of N-2 players,
        # leading to a very verbose output. We'll keep it to P1 vs P2 for simplicity here.

    def find_nash_equilibria(self):
        """Find pure strategy Nash equilibria."""
        print("\n--- Pure Strategy Nash Equilibrium Analysis ---")
        if not self.game:
            print("No game defined.")
            return

        players = self.game["players"]
        strategies = self.game["strategies"]
        payoffs = self.game["payoffs"]

        nash_equilibria = []

        # Generate all possible strategy profiles
        all_strategy_profiles_indices = list(product(*[range(len(s)) for s in strategies]))

        for profile_indices in all_strategy_profiles_indices:
            is_nash = True
            current_profile_strats = [strategies[p_idx][profile_indices[p_idx]] for p_idx in range(len(players))]

            for player_idx in range(len(players)):
                current_player_payoff = self._get_payoff_for_profile(player_idx, profile_indices)

                # Check if this player can unilaterally deviate and improve their payoff
                can_deviate = False
                for own_strat_idx_alt in range(len(strategies[player_idx])):
                    if own_strat_idx_alt == profile_indices[player_idx]:
                        continue # Skip current strategy

                    # Create a hypothetical new profile with the deviation
                    deviated_profile_indices = list(profile_indices)
                    deviated_profile_indices[player_idx] = own_strat_idx_alt

                    deviated_payoff = self._get_payoff_for_profile(player_idx, deviated_profile_indices)

                    if deviated_payoff > current_player_payoff:
                        can_deviate = True
                        break # Player can improve by deviating

                if can_deviate:
                    is_nash = False
                    break # Not a Nash Equilibrium if any player can deviate

            if is_nash:
                nash_equilibria.append(current_profile_strats)

        if nash_equilibria:
            print("Pure Strategy Nash Equilibria:")
            for eq in nash_equilibria:
                eq_str = ", ".join([f"{players[i]} plays '{eq[i]}'" for i in range(len(players))])
                print(f"- ({eq_str})")
        else:
            print("No pure strategy Nash equilibria found.")

    def _get_payoff_for_profile(self, player_idx, profile_indices):
        """Helper to get payoff for a given player and a full strategy profile (list of strategy indices)."""
        payoff_matrix = self.game["payoffs"][player_idx]
        current_payoff = payoff_matrix
        for dim_idx, strat_idx in enumerate(profile_indices):
            try:
                current_payoff = current_payoff[strat_idx]
            except IndexError:
                # This can happen if payoff matrix is not perfectly aligned with strategies,
                # or if strategies were eliminated. This function assumes original indices.
                # For `find_dominated_strategies` (iterative), we need to be careful with indices.
                # Here, for Nash Equilibrium, we assume we're looking at the original full matrix.
                print(f"Error: Payoff lookup failed for player {player_idx} at profile {profile_indices}. "
                      "Ensure payoff matrix dimensions match strategy lengths.")
                return 0 # Or raise an error
        return current_payoff

    def calculate_mixed_strategy_equilibrium(self):
        """
        Calculates mixed strategy Nash equilibrium.
        Currently supports only 2x2 games analytically.
        """
        print("\n--- Mixed Strategy Nash Equilibrium ---")
        if not self.game:
            print("No game defined.")
            return

        players = self.game["players"]
        strategies = self.game["strategies"]
        payoffs = self.game["payoffs"]

        if len(players) != 2 or len(strategies[0]) != 2 or len(strategies[1]) != 2:
            print("Mixed Strategy Nash Equilibrium calculation (analytical) is currently supported only for 2x2 games.")
            print("For larger games, iterative methods or linear programming are typically required.")
            return

        print("\nCalculating for 2x2 game:")
        # Player 1's strategies: s1_1, s1_2 (p, 1-p)
        # Player 2's strategies: s2_1, s2_2 (q, 1-q)

        # Payoffs for Player 1 (U1):
        # [[U1(s1_1, s2_1), U1(s1_1, s2_2)],
        #  [U1(s1_2, s2_1), U1(s1_2, s2_2)]]
        a, b = payoffs[0][0][0], payoffs[0][0][1]
        c, d = payoffs[0][1][0], payoffs[0][1][1]

        # Payoffs for Player 2 (U2):
        # [[U2(s1_1, s2_1), U2(s1_1, s2_2)],
        #  [U2(s1_2, s2_1), U2(s1_2, s2_2)]]
        e, f = payoffs[1][0][0], payoffs[1][0][1]
        g, h = payoffs[1][1][0], payoffs[1][1][1]

        # Player 1 is indifferent if Expected Payoff from Player 2's s2_1 == Expected Payoff from Player 2's s2_2
        # For Player 2 to be indifferent, Player 1's expected payoffs must be equal:
        # p * a + (1-p) * c = p * b + (1-p) * d
        # p * (a - b - c + d) = d - c
        # p = (d - c) / (a - b - c + d)

        # Player 2 is indifferent if Expected Payoff from Player 1's s1_1 == Expected Payoff from Player 1's s1_2
        # For Player 1 to be indifferent, Player 2's expected payoffs must be equal:
        # q * e + (1-q) * f = q * g + (1-q) * h
        # q * (e - f - g + h) = h - f
        # q = (h - f) / (e - f - g + h)

        p1_denom = (a - b - c + d)
        p2_denom = (e - f - g + h)

        p = None
        q = None

        if p1_denom != 0:
            p = (d - c) / p1_denom
        else:
            print(f"Warning: Player 1's denominator for mixed strategy is zero. Could indicate multiple mixed equilibria or no unique one based on indifference for Player 2.")

        if p2_denom != 0:
            q = (h - f) / p2_denom
        else:
            print(f"Warning: Player 2's denominator for mixed strategy is zero. Could indicate multiple mixed equilibria or no unique one based on indifference for Player 1.")

        print(f"\nPlayer 1's strategies: {strategies[0][0]} (p), {strategies[0][1]} (1-p)")
        print(f"Player 2's strategies: {strategies[1][0]} (q), {strategies[1][1]} (1-q)")

        if p is not None and 0 <= p <= 1:
            print(f"\nPlayer 1 should play '{strategies[0][0]}' with probability p = {p:.4f}")
            print(f"Player 1 should play '{strategies[0][1]}' with probability 1-p = {1-p:.4f}")
        else:
            print("\nNo interior mixed strategy for Player 1 (pure strategy might be optimal for P1).")
            # If p is outside [0,1], it means one of Player 1's pure strategies is always a best response for Player 2.

        if q is not None and 0 <= q <= 1:
            print(f"\nPlayer 2 should play '{strategies[1][0]}' with probability q = {q:.4f}")
            print(f"Player 2 should play '{strategies[1][1]}' with probability 1-q = {1-q:.4f}")
        else:
            print("\nNo interior mixed strategy for Player 2 (pure strategy might be optimal for P2).")
            # If q is outside [0,1], it means one of Player 2's pure strategies is always a best response for Player 1.

        if p is not None and q is not None and 0 <= p <= 1 and 0 <= q <= 1:
            # Calculate expected payoffs at the mixed strategy equilibrium
            exp_payoff_p1 = (p * q * a) + (p * (1-q) * b) + ((1-p) * q * c) + ((1-p) * (1-q) * d)
            exp_payoff_p2 = (p * q * e) + (p * (1-q) * f) + ((1-p) * q * g) + ((1-p) * (1-q) * h)
            print(f"\nExpected Payoff for {players[0]} at Mixed Nash Equilibrium: {exp_payoff_p1:.2f}")
            print(f"Expected Payoff for {players[1]} at Mixed Nash Equilibrium: {exp_payoff_p2:.2f}")

    def run_analysis(self):
        """Runs the complete game theory analysis based on user input."""
        print("\n=== Game Theory Analyzer ===")

        while True:
            print("\n--- Main Menu ---")
            print("1. Display Normal Form")
            print("2. Display Extensive Form (for 2-player simultaneous games)")
            print("3. Find Dominant/Dominated Strategies (and Rationalizable Strategies)")
            print("4. Find Best Responses")
            print("5. Find Pure Strategy Nash Equilibria")
            print("6. Calculate Mixed Strategy Nash Equilibrium (for 2x2 games)")
            print("7. Redefine Game")
            print("8. Exit")

            choice = input("Enter your choice: ")

            if choice == '1':
                self.display_normal_form()
            elif choice == '2':
                self.display_extensive_form()
            elif choice == '3':
                iterative_choice = input("Perform iterated elimination of strictly dominated strategies? (yes/no): ").lower()
                self.find_dominated_strategies(iterative=(iterative_choice == 'yes'))
            elif choice == '4':
                self.find_best_responses()
            elif choice == '5':
                self.find_nash_equilibria()
            elif choice == '6':
                self.calculate_mixed_strategy_equilibrium()
            elif choice == '7':
                self._get_user_game_input()
            elif choice == '8':
                print("Exiting Game Theory Analyzer. Goodbye!")
                break
            else:
                print("Invalid choice. Please try again.")

if __name__ == "__main__":
    analyzer = GameTheoryAnalyzer()
    analyzer.run_analysis()

--- Define Your Game ---
Enter the number of players (currently supports 2 for extensive form): 2
Enter name for Player 1: p1
Enter name for Player 2: p2
Enter strategies for p1 (comma-separated, e.g., Heads,Tails): H , T
Enter strategies for p2 (comma-separated, e.g., Heads,Tails): H , T

Enter payoffs for each player. For each cell (Player 1 Strategy, Player 2 Strategy), enter payoff for each player.
Example: If Player 1 plays 'Heads' and Player 2 plays 'Heads', enter 'P1_Payoff,P2_Payoff'.
Enter payoffs for (H, H) as 'P1_Payoff,P2_Payoff': 1, 1
Enter payoffs for (H, T) as 'P1_Payoff,P2_Payoff': 1,0
Enter payoffs for (T, H) as 'P1_Payoff,P2_Payoff': 0,1
Enter payoffs for (T, T) as 'P1_Payoff,P2_Payoff': 0,0

Game defined successfully!

=== Game Theory Analyzer ===

--- Main Menu ---
1. Display Normal Form
2. Display Extensive Form (for 2-player simultaneous games)
3. Find Dominant/Dominated Strategies (and Rationalizable Strategies)
4. Find Best Responses
5. Find Pure Strategy Nash E